In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [16]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:33<00:00, 42.67s/it]


In [5]:
len(deals)

50

In [6]:
deals[0]

<Oukitel BP2000 PRO 3,600W 2,048Wh Portable Power Station for $999 + free shipping>

In [8]:
print(deals[23].describe())

Title: Walmart Deals Week Automotive Sale: Up to 80% off + free shipping w/ $35
Details: Save on tire inflators, repair kits, mini dirt bikes, cleaning kits and much more. We've picked out this FRP GMB100 Gas Powered 99cc Mini Dirt Bike for $450 ($70 off.) Shipping adds $6.99, but orders of $35 or more get free shipping. Shop Now at Walmart
Features: 
URL: https://www.dealnews.com/Walmart-Deals-Week-Automotive-Sale-Up-to-80-off-free-shipping-w-35/21750828.html?iref=rss-c238


In [9]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [10]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [11]:
print(user_prompt)

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Oukitel BP2000 PRO 3,600W 2,048Wh Portable Power Station for $999 + free shipping
Details: Apply coupon code "Prime500" for a savings of $900. Buy Now at us.oukitel.com
Features: 3,300W AC output Max AC2200W fast charging Up to 10 year lifespan LiFePO4 battery 6 ways to charge (AC, Solar, AC+Solar, Car, Generator, and AC Adapter B;uetooth, WiFi, Smart app control Ex

In [13]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Oukitel BP2000 PRO 3,600W 2,048Wh Portable Power Station for $999 + free shipping
Details: Apply coupon code "Prime500" for a savings of $900. Buy Now at us.oukitel.com
Features: 3,300W AC output Max AC2200W fast charging Up to 10 year lifespan LiFePO4 battery 6 ways to charge (AC, Solar, AC+Solar, Car, Generator, and AC Adapter B;uetooth, WiFi, Smart app control Ex

In [14]:
def get_recommendations():
    completion = openai.chat.completions.parse(
        model="gpt-4.1-mini",
        messages = [
            {"role":"system","content":system_prompt},
            {"role":"user","content": user_prompt}
        ],
        response_format=DealSelection
    )
    response = completion.choices[0].message.parsed
    return response

In [17]:
result = get_recommendations()

In [21]:
len(result.deals)

5

In [23]:
print(result.deals)

[Deal(product_description='The Oukitel BP2000 PRO is a portable power station featuring a large 2,048Wh LiFePO4 battery with an extended lifespan of up to 10 years. It provides a maximum AC output of 3,300W and supports fast charging at 2,200W. The device can be charged through multiple methods including AC, solar, car, generator, and its own AC adapter. It supports smart app control via Bluetooth and WiFi and is expandable with up to seven additional expansion batteries for increased capacity.', price=999.0, url='https://www.dealnews.com/Oukitel-BP2000-PRO-3-600-W-2-048-Wh-Portable-Power-Station-for-999-free-shipping/21753698.html?iref=rss-c142'), Deal(product_description='The Vtoman FlashSpeed 1500 is a portable power station boasting a capacity of 1,548Wh. It offers three 110V AC outlets with 1,500W output capacity, alongside multiple USB ports including four USB-A and two USB-C ports, plus two DC5521 output ports and a car cigarette lighter port. Designed to provide versatile power

In [28]:
result.deals[2]

Deal(product_description='The Eco-Worthy 5.12kWh 48V 100AH Server Rack LiFePO4 Lithium Battery is designed for integration into standard 3U cabinets for server or storage use. It features a 100A battery management system (BMS) for safety and performance, wireless connectivity for smart monitoring, making it suitable for energy storage solutions with high capacity and intelligent management.', price=800.0, url='https://www.dealnews.com/Eco-Worthy-5-12-k-Wh-48-V-100-AH-Server-Rack-Li-Fe-PO4-Lithium-Battery-for-800-free-shipping/21753341.html?iref=rss-c142')

In [29]:
from agents.scanner_agent import ScannerAgent

In [30]:
agent = ScannerAgent()

In [31]:
result = agent.scan()

In [32]:
result

DealSelection(deals=[Deal(product_description='The Oukitel BP2000 PRO is a powerful portable power station that can deliver an impressive 3,600W with a maximum output of 3,300W. It features a robust 2,048Wh LiFePO4 battery that promises a lifespan of up to ten years, ensuring you have a reliable power source for all your needs. This unit offers six different charging methods, including AC, solar, and car options, and can be monitored and controlled via Bluetooth and smart app technology. Additionally, the power station is expandable to accommodate up to seven battery expansions, making it an excellent choice for extended off-grid adventures.', price=999.0, url='https://www.dealnews.com/Oukitel-BP2000-PRO-3-600-W-2-048-Wh-Portable-Power-Station-for-999-free-shipping/21753698.html?iref=rss-c142'), Deal(product_description='The Vtoman FlashSpeed 1500 Portable Power Station is a versatile and compact energy source, boasting a capacity of 1,548Wh and delivering up to 1,500W of continuous po